In [22]:
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pymongo
import scipy
import numpy as np
from db_interface import DbInterface,DbRecord

In [2]:
def getDbName(datasetName, modelName):
    return datasetName + '_' + modelName

In [3]:
def createSvmModel():
    return svm.SVC(kernel='linear')

In [7]:
def transform(features, trType=None):
    tr = lambda x: x
    if trType == 'pca':
        tr = lambda features: PCA(n_components=0.9).fit(features)
    elif trType == 'logm':
        def matrixLog(features): 
            res = np.tril(scipy.linalg.logm(features @ features.T))
            return res[res != 0]
        tr = matrixLog
            
    return [tr(feature.flatten()) for feature in features]

In [28]:
def train_svm(svmModel, client, datasetName, modelName):    
    dbCursor = client.getDB(getDbName(datasetName, modelName))
    layerNames = dbCursor.getCollectionNames()

    for layer in layerNames:
        records = [dbCursor.getRecord(layer, recordId) for recordId in dbCursor.getRecordIds(layer)]
        obs = [x.feature for x in records]
        target = [x.animalId for x in records]

        samples = transform(obs)
        X_train, X_test, y_train, y_test = train_test_split(samples, target, test_size=0.25)
        print('Training Size: {}, Test Size: {}'.format(len(X_train), len(X_test)))
        svmModel.fit(X_train, y_train)
        
        y_pred = svmModel.predict(X_test)
        print('Layer Name: {}, Accuracy: {}'.format(layer, metrics.accuracy_score(y_test, y_pred)))

In [29]:
modelNames = ['alexnet', 'googlenet']
datasetNames = ['amur']
client = DbInterface()

for dsName,modelName in [(x,y) for x in datasetNames for y in modelNames]:
    model = createSvmModel()
    print('Created an SVM for model {}, dataset {}'.format(modelName, dsName))
    train_svm(model, client, dsName, modelName)

Created an SVM for model alexnet, dataset amur
Training Size: 3, Test Size: 2
Layer Name: pool5, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: prob, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: relu4, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: drop7, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: relu6, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: relu3, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: relu7, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: conv4, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: conv3, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: fc8, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: relu1, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: norm2, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: pool1, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: conv2, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: relu2, Accuracy: 

Layer Name: inception_4b/relu_3x3_reduce, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: inception_3a/pool_proj, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: conv1/relu_7x7, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: inception_5a/3x3_reduce, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: inception_4a/relu_1x1, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: inception_3b/relu_5x5_reduce, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: pool2/3x3_s2, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: inception_3b/pool, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: inception_4b/3x3, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: inception_3a/pool, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: inception_4a/5x5_reduce, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: inception_4d/1x1, Accuracy: 0.0
Training Size: 3, Test Size: 2
Layer Name: inception_3a/relu_1x1, Accuracy: 0.0
Traini

NameError: name 'svmModel' is not defined